In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from topo import SHex
from mesh import Mesh2D, uniform_nodes_3d
from basis import LagrangeBasisHex

In [ ]:
order = 2
L = (1.0,2.0,3.0)
n_elem = 2
vertices, elem_to_vertex, boundary_vertices = \
          uniform_nodes_3d(n_elem,L[0],L[1],L[2])

In [ ]:
topo = SHex()
basis = LagrangeBasisHex(topo, order)

In [ ]:
mesh = Mesh2D(topo, basis)
mesh.build_mesh(vertices, elem_to_vertex, boundary_vertices)

In [ ]:
mesh.elem_to_dof